In [13]:
# check GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# connect to your google drive
from google.colab import drive
from pathlib import Path
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'yolov4/'
path = Path(base_dir + 'deploy_yolov4_on_video')
dest = path
dest.mkdir(parents=True, exist_ok=True)

In [12]:
# clone the darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [11]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [10]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [9]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [8]:
#download pre-trained weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [2]:
# upload video here
upload()

In [3]:
# upload config file here
upload()

In [4]:
# upload YOLO weight here
upload()

In [5]:
# upload obj.name here
upload()

In [1]:
# check files
!ls

In [6]:
# make inference here
!./darknet detector demo your_obj.data your_yolov4.cfg your_yolov4_best.weights -dont_show YOUR_Video.mp4 -i 0 -out_filename validation_results.avi

In [7]:
# add a threhold flag to show only detections with a confidence that is above the threhold
!./darknet detector demo your_obj.data your_yolov4.cfg your_yolov4_best.weights -thresh 0.60 -dont_show YOUR_Video.mp4 -i 0 -out_filename validation_results_thred60.avi